In [16]:
# Importing packages
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import numpy as np
from ipywidgets import Layout
import seaborn as sns
import math

import time
from scipy import linalg
from scipy import optimize
import sympy as sm

# local module for linear algebra
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Straight out copied from Monty P. 

## Importing and cleaning the data 

filename = 'imdb.csv'

## Defining the function to import and clean the data
def gen_df(filename):
    """ Returns a pandas dataframe containing information about movies from the IMDB database 

    Args:
    filename (csv-file): The csv-file have to be the file imdb.csv

    Returns:
    A pandas dataframe

    Notice:
    The function will not work if other files than imdb.csv are used as input """
        
    # Get .csv.file
    data = pd.read_csv(filename, sep=';', encoding='latin-1', escapechar='\\')

    # Read file into pandas dataframe
    df = pd.DataFrame(data)

    # Drop unwanted columns
    for i in range(44,48):
        df.drop(columns=[f'Unnamed: {i}'], inplace=True)
    df.drop(columns=['fn','wordsInTitle','url'], inplace=True)

    # Keep only observations of 'movie'-type
    I = df['type'] == 'video.movie'
    df = df.loc[I]
    df.drop(columns=['type'], inplace=True)

    # Drop movies with missing observations
    df.dropna(inplace=True)

    # During the importing process, the first decimal has become 0 for all movies.
    # Thus, we replace "0"'s with ""
    df['imdbRating'] = df['imdbRating'].astype(str)
    df['imdbRating'].replace(regex=True, inplace=True,to_replace='0',value='')
    df['imdbRating'] = df['imdbRating'].astype(float)

    # Transform duration from seconds to hours
    df['duration'] = df['duration']/60**2

    # Drop years before 1920 and the year 2014 because of few obervations
    I = (df['year']>=1920) & (df['year']<=2013)
    df = df.loc[I]

    # Change the type of 'year' to integer
    df['year'] = df['year'].astype(int)    
    
    # Sort observations and reset index
    df.sort_values('year', inplace=True)
    df.reset_index(inplace=True)

    # Generating a variable that shows the decade (as a string)
    year_list = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010]

    df['decade'] = ''
    for i,start in enumerate(year_list):
        end = start+10
        df.loc[(df['year'] >= start) & (df['year'] < end), 'decade'] = f'{year_list[i]}s'

    return df

df = gen_df(filename)



FileNotFoundError: File b'imdb.csv' does not exist

In [51]:
# Defining symbols 
sm.init_printing(use_unicode=True) # for pretty printing

g1 = sm.symbols("G_1") 
g2 = sm.symbols("G_2") 
g3 = sm.symbols("G_3") 
Gi = sm.symbols("G_i")
Dt = sm.symbols("D_t")
Dt1 = sm.symbols("D_t-1")
Dt2 = sm.symbols("D_t+1")
alpha1 = sm.symbols("alpha_1")
alpha2 = sm.symbols("alpha_2")
alpha3 = sm.symbols("alpha_3")
alphai = sm.symbols("alpha_i")
alpha = sm.symbols("alpha")
beta = sm.symbols("beta")
delta = sm.symbols("delta")
R = sm.symbols("R")
T = sm.symbols("T")



In [53]:
# Utility function

test = sm.Eq(1/2 * g1**alpha1 + g2**alpha2 + g3**alpha3,T)

utility = sm.Eq(alpha * (alpha1*g1 + alpha2*g2 + alpha3*g3) + beta*(1/2 * Dt1 + Dt + 1/2 * Dt2) + delta*R, T)
utility

In [ ]:
# Define user 